<a href="https://colab.research.google.com/github/azeem110201/fake_news_detector/blob/master/Fake_news_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
dataset = '/content/drive/My Drive/news.zip (Unzipped Files)/news.csv'

In [0]:
import pandas as pd
df = pd.read_csv(dataset)

In [0]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [0]:
len(df)

6335

In [0]:
6335 - 1250

5085

In [0]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [0]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [0]:
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [0]:
for i in range(0,10):
    print(df['title'][i])

You Can Smell Hillary’s Fear
Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)
Kerry to go to Paris in gesture of sympathy
Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!'
The Battle of New York: Why This Primary Matters
Tehran, USA
Girl Horrified At What She Watches Boyfriend Do After He Left FaceTime On
‘Britain’s Schindler’ Dies at 106
Fact check: Trump and Clinton at the 'commander-in-chief' forum
Iran reportedly makes new push for uranium concessions in nuclear talks


In [0]:
print(df['title'][0])

You Can Smell Hillary’s Fear


In [0]:
print(df['text'][0])

Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. 
In the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. 
The word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. 
But that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. 
The FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. 
The FBI’s leadership is being warned that the entire left

In [0]:
df['label'] = df['label'].map({'REAL':1,'FAKE':0})

In [0]:
df['label']

0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Name: label, Length: 6335, dtype: int64

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
vocab_len = 20000
trunc_type = 'pre'
padding_type = 'pre'
embedding_layer = 16
max_length = 1080
OOV_token = '<OOV>'

In [0]:
features = list(df['text'])
labels = list(df['label'])

In [0]:
training_feature = features[:5085]
testing_feature = features[5085:]
training_label = labels[:5085]
testing_label = labels[5085:]

In [0]:
tokenizer = Tokenizer(num_words=vocab_len,oov_token=OOV_token)

In [0]:
tokenizer.fit_on_texts(training_feature)

In [0]:
word_index = tokenizer.word_index

In [0]:
sequence = tokenizer.texts_to_sequences(training_feature)

In [0]:
sequence = pad_sequences(sequences=sequence,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [0]:
sequence.shape

(5085, 1080)

In [0]:
sequence

array([[ 245, 1176,    5, ...,  406,  828,   25],
       [   0,    0,    0, ..., 1196, 1209,  539],
       [   0,    0,    0, ...,    4, 6723,   22],
       ...,
       [4899,   12,    1, ...,    1,   29,  190],
       [   0,    0,    0, ...,   26,  266, 2881],
       [   0,    0,    0, ..., 1984,   91,  332]], dtype=int32)

In [0]:
import numpy as np

training_label = np.array(training_label)
testing_label = np.array(testing_label)

In [0]:
test_sequence = tokenizer.texts_to_sequences(testing_feature)
test_sequence = pad_sequences(sequences=test_sequence,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [0]:
test_sequence.shape

(1250, 1080)

In [0]:
test_sequence

array([[    0,     0,     0, ...,    38,   913,  7989],
       [    0,     0,     0, ...,     5,    24,   580],
       [    0,     0,     0, ...,   909,     7,  7382],
       ...,
       [ 9942,   295,   625, ...,     3,   116,   372],
       [   29,  7042,     1, ...,    17,  1078,    22],
       [    0,     0,     0, ..., 13383,     2,  7849]], dtype=int32)

In [0]:
import tensorflow as tf

In [0]:
def create_model():

    model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(input_dim=vocab_len,output_dim=embedding_layer,input_length=max_length),
                             tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32,kernel_initializer='he_normal',return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,kernel_initializer='he_normal',return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,kernel_initializer='he_normal',return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128,kernel_initializer='he_normal',return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256,kernel_initializer='he_normal')),
                             tf.keras.layers.Dense(128,activation='relu'),
                             tf.keras.layers.Dense(256,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
    ])

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [0]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1080, 16)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 1080, 64)          9600      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1080, 128)         49920     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1080, 128)         74496     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1080, 256)         198144    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 512)               789504    
_________________________________________________________________
dense (Dense)                (None, 128)               6

In [0]:
epoch = 5

history = model.fit(sequence,training_label,epochs=epoch,validation_data=(test_sequence,testing_label),batch_size=32,validation_batch_size=32)

Epoch 1/5
159/159 [==============================] - 2128s 13s/step - loss: 0.5226 - accuracy: 0.7261 - val_loss: 0.4346 - val_accuracy: 0.7840
Epoch 2/5
159/159 [==============================] - 2146s 13s/step - loss: 0.2158 - accuracy: 0.9148 - val_loss: 0.4620 - val_accuracy: 0.8312
Epoch 3/5
159/159 [==============================] - 2143s 13s/step - loss: 0.0523 - accuracy: 0.9821 - val_loss: 0.5982 - val_accuracy: 0.8200
Epoch 4/5
159/159 [==============================] - 2128s 13s/step - loss: 0.0126 - accuracy: 0.9953 - val_loss: 1.0418 - val_accuracy: 0.8280
Epoch 5/5
159/159 [==============================] - 2148s 14s/step - loss: 0.0257 - accuracy: 0.9906 - val_loss: 0.8184 - val_accuracy: 0.8296


In [0]:
 !pip install pyyaml h5py

In [0]:
model.save('fake_news_detector_trained_model.h5') 

In [0]:
new_model = tf.keras.models.load_model('/content/drive/My Drive/fake_news_detector_trained_model.h5')

In [0]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1080, 16)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 1080, 64)          9600      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1080, 128)         49920     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1080, 128)         74496     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1080, 256)         198144    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 512)               789504    
_________________________________________________________________
dense (Dense)                (None, 128)               6

In [0]:
test_text = ['Trump is going to call economic recession where all the people in the world would suffer from hunger and starvation.Many leaders in the world are in support of this specially Modi.World economy is going to trash this year because of coronavirus']

In [0]:
sequence_to_test = tokenizer.texts_to_sequences(test_text)
sequence_to_test = pad_sequences(sequence_to_test,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [0]:
sequence_to_test

array([[ 0,  0,  0, ..., 89,  4,  1]], dtype=int32)

In [0]:
pred = model.predict(sequence_to_test)

In [0]:
pred[0]

array([0.00021642], dtype=float32)